In [3]:
import pandas as pd
import numpy as numpy
import bs4 
import requests 
import spotipy


In [31]:
data=pd.read_csv('del91.csv')

In [38]:
data.head()

,Unnamed: 0,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,0,House Of Lies,"Ikka, Karan Aujla, Sanjoy",House Of Lies,58U6KMHr9ApwuTXeKXIGa9,7avo2LpNFXrgUgRtl9OsAL,61,2024-05-24,179170,False,...,0.606,4,-6.551,0,0.183,0.1470,0.000000,0.1370,0.401,139.341
1,1,Up To U,Dhanda Nyoliwala,Up To U,3LqdYNb1s5tt77gpYkMeeF,7695jWrVLrftHb9ChEmFCg,51,2023-05-06,188653,False,...,0.565,9,-8.990,1,0.259,0.0711,0.000000,0.0827,0.496,142.989
2,2,Matha Garam,"DG IMMORTALS, Raga",Matha Garam,0rUPO4MVsnME1dyAmmgHm9,6v4dEkFs3I2ImanEVok2cP,41,2024-05-24,206000,True,...,0.780,7,-4.795,1,0.230,0.2450,0.000000,0.1110,0.733,104.998
3,3,Ballaatha Jaathi,"NJ, Dabzee, BABY JEAN, Rzee",Ballaatha Jaathi,2lVMOwZzF8iqaPbO83XScW,1LBSyVTSeYbgBzV7AznMsN,72,2024-03-26,230000,False,...,0.616,1,-7.088,0,0.139,0.2350,0.000486,0.3860,0.858,95.969
4,4,Dupatta Drill,"Simiran Kaur Dhadli, Rawme Hooda, Pranjal Dahiya",Dupatta Drill,545u6lIVWOfi58QE5I3feD,6wwwaMSY04xYhDpU7IKzz3,34,2024-05-22,150339,False,...,0.690,0,-6.079,1,0.239,0.3160,0.000000,0.1350,0.546,92.966


In [32]:
data.isnull().sum()

Unnamed: 0          0
Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
def calculate_weighted_popularity(release_date):
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    time_span = datetime.now() - release_date
    weight = 1 / (time_span.days + 1)
    return weight

In [36]:
scaler = MinMaxScaler()
music_features = data[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [37]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in data['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return
    input_song_index = data[data['Track Name'] == input_song_name].index[0]
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]
    content_based_recommendations = data.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]
    return content_based_recommendations

In [43]:
input_song_name = "TRAP PRAA"
recommendations =content_based_recommendations(input_song_name, num_recommendations=7)
print(f"content based recommendations songs for '{input_song_name}':")
print(recommendations)

content based recommendations songs for 'TRAP PRAA':
       Track Name                                     Artists     Album Name  \
20      King Shit                                       Shubh            Leo   
34  Rubicon Drill  Laddi Chahal, Parmish Verma, Gurlez Akhtar        Forever   
43     Bt Ho Gayi                                     Paradox     Hustle 2.0   
38        Ram Ram                                   MC SQUARE     Hustle 2.0   
32         On Top                                 Karan Aujla         On Top   
42  The Last Ride               Sidhu Moose Wala, Wazir Patar  The Last Ride   
46           Daku                 Chani Nattan, Inderpal Moga           Daku   

   Release Date  Popularity  
20   2024-01-05          77  
34   2022-11-16          71  
43   2022-10-09           0  
38   2022-10-09           0  
32   2022-11-25          72  
42   2022-05-15          66  
46   2021-01-16          72  
